In [5]:
!pip install qiskit qiskit_aer 

  Using cached qiskit_aer-0.17.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [1]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
from tqdm import tqdm

In [2]:
def amplitude_encode(features):
    d = len(features)
    n_qubits = int(np.ceil(np.log2(d)))
    dim = 2**n_qubits
    vec = np.zeros(dim)
    vec[:d] = features
    norm = np.linalg.norm(vec)
    return vec / norm if norm != 0 else vec

def build_state_prep(vec, n_qubits):
    qc = QuantumCircuit(n_qubits)
    qc.initialize(vec, range(n_qubits))
    return qc

def swap_test(vec1, vec2, shots=1024):
    n_qubits = int(np.log2(len(vec1)))
    qc = QuantumCircuit(1 + 2 * n_qubits, 1)
    qc.h(0)

    # Encode both vectors
    qc.compose(build_state_prep(vec1, n_qubits), qubits=range(1, 1 + n_qubits), inplace=True)
    qc.compose(build_state_prep(vec2, n_qubits), qubits=range(1 + n_qubits, 1 + 2 * n_qubits), inplace=True)

    # Controlled-SWAP between registers
    for i in range(n_qubits):
        qc.cswap(0, i + 1, i + 1 + n_qubits)

    qc.h(0)
    qc.measure(0, 0)

    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    prob_0 = counts.get('0', 0) / shots
    fidelity = 2 * prob_0 - 1
    return fidelity


In [3]:
df = pd.read_csv('data.csv')

# Initialize empty lists for the training and testing data
X_train = []
y_train = []
X_test = []
y_test = []

# Iterate over each label
labels = df['label'].unique()  # Get the unique labels
for label in labels:
    # Select the rows corresponding to the current label
    label_data = df[df['label'] == label]
    
    # Take the first 15 rows for training
    X_train.append(label_data[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].iloc[:15].values)
    y_train.append(label_data['label'].iloc[:15].values)
    
    # Take the next 5 rows for testing
    X_test.append(label_data[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']].iloc[15:20].values)
    y_test.append(label_data['label'].iloc[15:20].values)

# Convert lists to arrays
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

In [4]:
train_states = []
for xi in tqdm(X_train, desc="Encoding training"):
    train_states.append(amplitude_encode(xi))
train_labels = list(y_train)


Encoding training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 330/330 [00:00<00:00, 62277.63it/s]


In [5]:
k = 5
predictions = []
for xi in tqdm(X_test, desc="Predicting"):
    vec_test = amplitude_encode(xi)
    fidelities = [swap_test(vec_test, vec_train, shots=1024) for vec_train in train_states]
    distances = [1 - f for f in fidelities]
    nn_idx = np.argsort(distances)[:k]
    nearest = [train_labels[i] for i in nn_idx]
    predictions.append(Counter(nearest).most_common(1)[0][0])


Predicting: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [1:45:56<00:00, 57.79s/it]


In [6]:
acc = accuracy_score(y_test, predictions)
print(f"Accuracy: {acc:.2f}\n")
print(classification_report(y_test, predictions))
 

Accuracy: 0.89

              precision    recall  f1-score   support

       apple       1.00      1.00      1.00         5
      banana       0.83      1.00      0.91         5
   blackgram       0.71      1.00      0.83         5
    chickpea       1.00      1.00      1.00         5
     coconut       1.00      1.00      1.00         5
      coffee       1.00      0.80      0.89         5
      cotton       1.00      1.00      1.00         5
      grapes       1.00      1.00      1.00         5
        jute       0.75      0.60      0.67         5
 kidneybeans       0.83      1.00      0.91         5
      lentil       0.60      0.60      0.60         5
       maize       1.00      0.80      0.89         5
       mango       1.00      0.80      0.89         5
   mothbeans       0.50      0.20      0.29         5
    mungbean       1.00      1.00      1.00         5
   muskmelon       1.00      1.00      1.00         5
      orange       1.00      1.00      1.00         5
      papay